<!-- Autogenerated by `scripts/make_examples.py` -->
<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/voxel51/fiftyone-examples/blob/master/examples/evaluate_detections.ipynb">
            <img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32">
            Try in Google Colab
        </a>
    </td>
    <td>
        <a target="_blank" href="https://nbviewer.jupyter.org/github/voxel51/fiftyone-examples/blob/master/examples/evaluate_detections.ipynb">
            <img src="https://user-images.githubusercontent.com/25985824/104791634-6efa1d80-5769-11eb-8a4c-71d6cb53ccf0.png" height="32" width="32">
            Share via nbviewer
        </a>
    </td>
    <td>
        <a target="_blank" href="https://github.com/voxel51/fiftyone-examples/blob/master/examples/evaluate_detections.ipynb">
            <img src="https://user-images.githubusercontent.com/25985824/104791633-6efa1d80-5769-11eb-8ee3-4b2123fe4b66.png" height="32" width="32">
            View on GitHub
        </a>
    </td>
    <td>
        <a href="https://github.com/voxel51/fiftyone-examples/raw/master/examples/evaluate_detections.ipynb" download>
            <img src="https://user-images.githubusercontent.com/25985824/104792428-60f9cc00-576c-11eb-95a4-5709d803023a.png" height="32" width="32">
            Download notebook
        </a>
    </td>
</table>


# Evaluating Object Detections

This example provides a brief overivew of using FiftyOne to quantitatively and qualitatively evaluate object detections.

For more details, check out the in-depth
[evaluating object detections tutorial](https://voxel51.com/docs/fiftyone/tutorials/evaluate_detections.html).

## Load dataset

We'll work with the validation split of the [COCO dataset](https://cocodataset.org/#home), which is conveniently available in the [FiftyOne Dataset Zoo](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/zoo.html):

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

# Load the COCO validation split
# This will download the dataset from the web, if necessary
dataset = foz.load_zoo_dataset("coco-2017", split="validation")

print(dataset)

Split 'validation' already downloaded
Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Name:           coco-2017-validation
Media type:     None
Num samples:    5000
Persistent:     True
Info:           {'classes': ['0', 'person', 'bicycle', ...]}
Tags:           ['validation']
Sample fields:
    media_type:   fiftyone.core.fields.StringField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


## Adding model predictions

See [this tutorial section](https://voxel51.com/docs/fiftyone/tutorials/evaluate_detections.html#Add-predictions-to-dataset) to see how to easily add predictions from your model to a FiftyOne dataset.

In this example, we'll download some pre-computed predictions from the web and merge them into our FiftyOne dataset.

In [4]:
import os

import eta.core.utils as etau
import eta.core.web as etaw


def merge_json_dataset(dataset, fid):
    """Downloads a JSON-only export of a FiftyOne Dataset from Google
    Drive and merges its contents into the provided dataset.
    
    The samples are joined by basename of their `filepath`.
    
    Args:
        dataset: a :class:`fiftyone.core.dataset.Dataset`
        fid: the ID of a Google Drive file containing a dataset export
            generated via :meth:`fiftyone.core.dataset.Dataset.write_json`
    """
    with etau.TempDir() as tmp_dir:
        # Download dataset from Google Drive
        print("Downloading Google Drive file '%s'..." % fid)
        json_path = os.path.join(tmp_dir, "dataset.json")
        etaw.download_google_drive_file(fid, path=json_path)

        # Load as FiftyOne dataset
        print("Loading download as FiftyOne Dataset...")
        json_dataset = fo.Dataset.from_json(
            json_path, name=fo.get_default_dataset_name()
        )

    # Merge predictions into `dataset` by filepath
    print("Merging data into '%s'..." % dataset.name)
    dataset.merge_samples(
        json_dataset,
        key_field="filepath",
        key_fcn=lambda k: os.path.basename(k),
    )


In [5]:
# Merge predictions into dataset
merge_json_dataset(dataset, "1AoX6AHZeGvaHaYu6iKatCtLTeLwLSaf-")

print(dataset)

 100% |██████|    6.7Mb/6.7Mb [147.4ms elapsed, 0s remaining, 45.7Mb/s]     
Loading download as FiftyOne Dataset...
 100% |█████| 5000/5000 [4.9s elapsed, 0s remaining, 1.0K samples/s]       
Merging data into 'coco-2017-validation'...
 100% |█████| 5000/5000 [36.4s elapsed, 0s remaining, 142.3 samples/s]      
Name:           coco-2017-validation
Media type:     None
Num samples:    5000
Persistent:     True
Info:           {'classes': ['0', 'person', 'bicycle', ...]}
Tags:           ['validation']
Sample fields:
    media_type:   fiftyone.core.fields.StringField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


The dataset now contains predictions in its `faster_rcnn` field.

## Visualizing detections in the App

First, lets locate the samples in the dataset that have predictions:

In [6]:
# Get samples that have predictions in the `faster_rcnn` field
predictions_view = dataset.exists("faster_rcnn")

print(predictions_view)

Dataset:        coco-2017-validation
Media type:     None
Num samples:    15
Tags:           ['validation']
Sample fields:
    media_type:   fiftyone.core.fields.StringField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Pipeline stages:
    1. Exists(field='faster_rcnn', bool=True)


and visualize them in the App:

In [9]:
session = fo.launch_app(view=predictions_view)

App launched


![eval-01](https://user-images.githubusercontent.com/25985824/97063118-edc92b00-156b-11eb-9366-120f9d17857c.png)

## Evaluating detections

FiftyOne provdies a [utility method](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.eval.coco.html#fiftyone.utils.eval.coco.evaluate_detections) that performs COCO-style evaluation on the detections in a dataset/view:

In [12]:
import fiftyone.utils.eval as foue

foue.evaluate_detections(
    predictions_view, "faster_rcnn", gt_field="ground_truth"
)

Evaluating detections...
 100% |█████████| 15/15 [448.3ms elapsed, 0s remaining, 33.5 samples/s]      


Refreshing the view in the App, we see that additional fields have been added to each sample to tabulate the evaluation metrics:

In [13]:
# Update the view in the App
session.view = predictions_view

Both visually and quantitatively, we see that the model is generating too many false positive predictions!

![eval-03](https://user-images.githubusercontent.com/25985824/97063120-f1f54880-156b-11eb-96ac-8874aa8ff503.png)

![eval-04](https://user-images.githubusercontent.com/25985824/97063122-f28ddf00-156b-11eb-873d-cb22eebe8052.png)

## Filtering by confidence threshold

Use the label filters in the App to try different confidence thresholds for the `predictions` field of the dataset.

Alternatively, you can perform the same [filtering operation](https://voxel51.com/docs/fiftyone/user_guide/using_views.html#filtering) via Python code:

In [14]:
from fiftyone import ViewField as F

# Extract detections with confidence >= 0.80
high_conf_predictions_view = predictions_view.filter_labels(
    "faster_rcnn", F("confidence") > 0.80
)

print(high_conf_predictions_view)

Dataset:        coco-2017-validation
Media type:     None
Num samples:    15
Tags:           ['validation']
Sample fields:
    media_type:   fiftyone.core.fields.StringField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    tp_iou_0_75:  fiftyone.core.fields.IntField
    fp_iou_0_75:  fiftyone.core.fields.IntField
    fn_iou_0_75:  fiftyone.core.fields.IntField
Pipeline stages:
    1. Exists(field='faster_rcnn', bool=True)
    2. FilterDetections(field='faster_rcnn', filter={'$gt': ['$$this.confidence', 0.8]}, only_matches=False)


We can easily verify that the detections do, in fact, have high confidencee:

In [15]:
# Print a sample detection from the view
sample = high_conf_predictions_view.first()
print(sample.faster_rcnn.detections[0])

<Detection: {
    'id': '5f900c2673a342f845867a27',
    'attributes': BaseDict({}),
    'label': 'bird',
    'bounding_box': BaseList([
        0.537960205078125,
        0.6442485758463542,
        0.02527740478515625,
        0.0403411865234375,
    ]),
    'mask': None,
    'confidence': 0.9909082651138306,
    'index': None,
    'ground_truth_eval': BaseDict({
        'matches': BaseDict({
            '0_75': BaseDict({
                'gt_id': '5f935ec949742de0f5b8ab00',
                'iou': 0.7603734600073444,
            }),
        }),
    }),
}>


Don't worry, the lower confidence predictions have not been deleted! They are just being excluded from the view.

Let's re-run the evaluation method to update the metrics for the high-confidence-only predictions:

In [16]:
# Re-evaluate high confidence-only predictions
foue.evaluate_detections(
    high_conf_predictions_view, "faster_rcnn", gt_field="ground_truth"
)

session.view = high_conf_predictions_view

Evaluating detections...
 100% |█████████| 15/15 [237.0ms elapsed, 0s remaining, 65.5 samples/s]     


As we can now see, both visually and quantitatively, the false positive rate of the model has been decreased!

![eval-05](https://user-images.githubusercontent.com/25985824/97063124-f3267580-156b-11eb-93b9-f71a29c2d450.png)

![eval-06](https://user-images.githubusercontent.com/25985824/97063125-f3267580-156b-11eb-95fb-bf12bc17446e.png)

## More advanced filtering operations

The examples below demonstrate some more advanced [filtering operations](https://voxel51.com/docs/fiftyone/user_guide/using_views.html#filtering) that you can perform via dataset views:

### Small vs large detections

Let's create a view that only contains small bounding boxes; i.e., those with area less than 0.01 (less than 1% of the image):

In [17]:
# Bounding boxes are stored in [top-left-x, top-left-y, width, height] format
# with relative coordinates in [0, 1] x [0, 1]
bbox_area = F("bounding_box")[2] * F("bounding_box")[3]

# Create a view that only contains detections with area < 0.01, and 
# only contains samples with at least one detection after filtering
small_boxes_view = dataset.filter_labels(
    "ground_truth", bbox_area < 0.01, only_matches=True
)

session.view = small_boxes_view

![eval-07](https://user-images.githubusercontent.com/25985824/97063127-f3bf0c00-156b-11eb-8313-c4b36df733d2.png)

Now let's compare that to a view that only contains large bounding boxes:

In [18]:
# Create a view that only contains detections with area >= 0.01, and 
# only contains samples with at least one detection after filtering
large_boxes_view = dataset.filter_labels(
    "ground_truth", bbox_area >= 0.01, only_matches=True
)

session.view = large_boxes_view

![eval-08](https://user-images.githubusercontent.com/25985824/97063128-f457a280-156b-11eb-9de6-989fdbe30cfd.png)

### Filtering + sorting by label

Now let's show only `person` detections, and show samples with the most people
first:

In [19]:
# Create a view that only contains detections with label `person`, and
# show the images with the most people first
person_view = (
    dataset
    .filter_labels("ground_truth", F("label") == "person")
    .sort_by(F("ground_truth.detections").length(), reverse=True)
)

session.view = person_view

![eval-09](https://user-images.githubusercontent.com/25985824/97063129-f457a280-156b-11eb-9afe-66e517a73313.png)

![eval-10](https://user-images.githubusercontent.com/25985824/97063130-f4f03900-156b-11eb-91e1-99c81c5a5e54.png)